## SQLZoo Covid19 Window LAG Practice Solutions.

**1-**
* The example uses a WHERE clause to show the cases in 'Italy' in March 2020.
* **Modify the query to show data from Spain**.

In [ ]:

SELECT name, DAY(whn),
confirmed, deaths, recovered
FROM covid
WHERE name = 'Spain'
AND MONTH(whn) = 3 AND YEAR(whn) = 2020
ORDER BY whn
;
    

**2-**
* The LAG function is used to show data from the preceding row or the table. When lining up rows the data is partitioned by country name and ordered by the data whn. That means that only data from Italy is considered.
* **Modify the query to show confirmed for the day before**.

In [ ]:

SELECT name, DAY(whn), confirmed,
LAG(confirmed, 1) OVER (PARTITION BY name ORDER BY whn) AS day_b4_confirmed
FROM covid
WHERE name = 'Italy'
AND MONTH(whn) = 3 AND YEAR(whn) = 2020
ORDER BY whn
;
    

**3-**
* The number of confirmed case is cumulative - but we can use LAG to recover the number of new cases reported for each day
* **Show the number of new cases for each day, for Italy, for March**.

In [ ]:

SELECT name, DAY(whn), 
(
confirmed - LAG(confirmed, 1) OVER (PARTITION BY name ORDER BY whn)
)    AS daily_new
FROM covid
WHERE name = 'Italy'
AND MONTH(whn) = 3 AND YEAR(whn) = 2020
ORDER BY whn
;


**4-**
* The data gathered are necessarily estimates and are inaccurate. However by taking a longer time span we can mitigate some of the effects.
* You can filter the data to view only Monday's figures **WHERE WEEKDAY(whn) = 0**.
* **Show the number of new cases in Italy for each week in 2020 - show Monday only**.

In [ ]:

SELECT name, DATE_FORMAT(whn,'%Y-%m-%d') AS weekly_date, 
(
confirmed -
LAG(confirmed, 1) OVER (PARTITION BY name ORDER BY whn)
)    AS weekly_new

FROM covid
WHERE name = 'Italy'
AND WEEKDAY(whn) = 0 AND YEAR(whn) = 2020
ORDER BY whn
;


**5-**
* You can JOIN a table using DATE arithmetic. This will give different results if data is missing.
* **Show the number of new cases in Italy for each week - show Monday only.**.
* In the sample query we JOIN this week **tw** with last week **lw** using the DATE_ADD function.

In [ ]:

SELECT tw.name, 
DATE_FORMAT(tw.whn,'%Y-%m-%d') AS weekly_date, 
(tw.confirmed - lw.confirmed) AS weekly_new
FROM covid tw LEFT JOIN covid lw ON 
DATE_ADD(lw.whn, INTERVAL 1 WEEK) = tw.whn
AND tw.name=lw.name
WHERE tw.name = 'Italy'
AND WEEKDAY(tw.whn) = 0
ORDER BY tw.whn
;
    

**6-**
* This query shows the number of confirmed cases together with the world ranking for cases for the date '2020-04-20'. The number of COVID deaths is also shown.
* United States has the highest number, Spain is number 2...
* Notice that while Spain has the second highest confirmed cases, Italy has the second highest number of deaths due to the virus.
* **Add a column to show the ranking for the number of deaths due to COVID**.

In [ ]:

SELECT name, confirmed,
RANK() OVER (ORDER BY confirmed DESC) AS infections_rank,
deaths,
RANK() OVER (ORDER BY deaths DESC) AS deaths_rank
FROM covid
WHERE whn = '2020-04-20'
ORDER BY confirmed DESC
;
    

**7-**
* This query includes a JOIN t the world table so we can access the total population of each country and calculate infection rates (in cases per 100,000).
* **Show the infection rate ranking for each country. Only include countries with a population of at least 10 million**.

In [ ]:

SELECT 
world.name,
ROUND(100000*confirmed/population,0)as infect_rate,
RANK() OVER(ORDER BY infect_rate) 
FROM covid JOIN world ON covid.name=world.name
WHERE whn = '2020-04-20' AND population > 10000000
ORDER BY population DESC
;


**8-**
* **For each country that has had at last 1000 new cases in a single day, show the date of the peak number of new cases.**.

In [ ]:

WITH cte1 AS
(
SELECT *, (confirmed - LAG(confirmed, 1) OVER (PARTITION BY name ORDER BY whn)) AS daily_cases
FROM covid
)
, cte2 (name, max_cases) AS
(
SELECT name, MAX(daily_cases),
FROM cte1
GROUP BY name
HAVING max_cases >= 1000
)
SELECT cte2.name, DATE_FORMAT(covid.whn,'%Y-%m-%d') AS date, cte2.max_cases
FROM cte2 LEFT JOIN cte1 ON (cte2.name = cte1.name) AND (cte2.max_cases = cte1.daily_cases)
ORDER BY date_;
